<a href="https://colab.research.google.com/github/tantanchen/Stream/blob/main/Phi_1_5_with_JSONformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers
!pip install accelerate
!pip install einops
!pip install jsonformer

In [ ]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from jsonformer import Jsonformer

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Open-Orca/oo-phi-1_5",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
    ).to('cuda')
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/oo-phi-1_5",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
sys_prompt = "I am a data generator, I will generate unique and uncommon contact info consisting of names and address"
input = "Generate some unique name and addresses"

prefix = "<|im_start|>"
suffix = "<|im_end|>\n"
sys_format = prefix + "system\n" + sys_prompt + suffix
user_format = prefix + "user\n" + input + suffix
assistant_format = prefix + "assistant\n"
input_text = sys_format + user_format + assistant_format

In [ ]:
generation_config = GenerationConfig(
    max_length=1024, temperature=0.9, top_p=0.95, repetition_penalty=1.1,
    do_sample=True, use_cache=True,
    eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id)

inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=False).to('cuda')
outputs = model.generate(**inputs, generation_config=generation_config)

text = tokenizer.batch_decode(outputs)[0]
print(text)

<|im_start|>system
I am a data generator, I will generate unique and uncommon contact info consisting of names and address<|im_end|>
<|im_start|>user
Generate some unique name and addresses<|im_end|>
<|im_start|>assistant
Here are 10 unique names and addresses:

1. John Smith, 123 Main Street, New York City, NY 10001
2. Jane Doe, 456 Oak Avenue, Los Angeles, CA 90055
3. Bob Johnson, 789 Pine Road, Seattle, WA 98104
4. Mary Thompson, 321 Harbor Drive, London, UK 50207
5. Alex Lee, 567 Elm Boulevard, Sydney, Australia 57601
6. Sarah Kim, 890 Cedar Avenue, Tokyo, Japan Tōku 05001
7. Michael Lee, 987 Maple Street, San Francisco, CA 94117
8. Olivia Davis, 345 Birchwood Drive, Seoul, Kő, Republic of Korea 90242
9. David Brown, 678 Maple Lane, Paris, France 678431
10. Lisa Wilson, 908 Fifth Avenue, London, UK 75111

Please note that the street numbers might be slightly different as they are for locations in Europe or other regions not included in the list above. However, all these names have 

In [ ]:
bos_token = tokenizer.bos_token
eos_token = tokenizer.eos_token

print(bos_token)
print(eos_token)

<|endoftext|>
<|im_end|>


In [ ]:
json_schema = {
    "type": "object",
    "properties": {
        "first_name": {"type": "string"},
        "middle_name": {"type": "string"},
        "last_name": {"type": "string"},
        "street": {"type": "string"},
        "city": {"type": "string"},
        "state": {"type": "string"},
        "postal_code": {"type": "string"}

        }
    }


prompt = "Matthew L Keller and Julie Smith 983 Second Ave, Springfield, New York, 19283"
jsonformer = Jsonformer(model, tokenizer, json_schema, text)
generated_data = jsonformer()

print(json.dumps(generated_data, indent=4))

{
    "first_name": "John",
    "middle_name": "Smith",
    "last_name": "Smith",
    "street": "123 Main Street",
    "city": "New York City",
    "state": "NY",
    "postal_code": "10001"
}
